In [1]:
# inportar librerias
import sqlite3 # Módulo para trabajar con bases de datos SQLite3
import pandas as pd # Módulo para trabajar con estructuras de datos tipo DataFrame
# conectarse a la base de datos
conn = sqlite3.connect('data/db_movies')
cur = conn.cursor() # Crear un cursor para ejecutar sentencias SQL

# consultamos las tablas de la base de datos
cur.execute("SELECT name FROM sqlite_master WHERE type='table';")
print(cur.fetchall())

[('ratings',), ('movies',), ('movies_clean',), ('full_table',), ('data_clean',)]


In [2]:
# ver tabla de data_clean
df = pd.read_sql_query("SELECT * FROM data_clean", conn)
df.head()

,movieId,title,genres,userId,rating,timestamp
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,1,4.0,964982703
1,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,5,4.0,847434962
2,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,7,4.5,1106635946
3,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,15,2.5,1510577970
4,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,17,4.5,1305696483


In [5]:
###################################################################################
###################################################################################
####codigo para separar género #############################################
##########################################################################################
from mlxtend.preprocessing import TransactionEncoder

genres=df['genres'].str.split('|')
te = TransactionEncoder()
genres = te.fit_transform(genres)
genres = pd.DataFrame(genres, columns = te.columns_)
genres.head()

,Action,Adventure,Animation,Children,Comedy,Crime,Documentary,Drama,Fantasy,Film-Noir,Horror,IMAX,Musical,Mystery,Romance,Sci-Fi,Thriller,War,Western
0,False,True,True,True,True,False,False,False,True,False,False,False,False,False,False,False,False,False,False
1,False,True,True,True,True,False,False,False,True,False,False,False,False,False,False,False,False,False,False
2,False,True,True,True,True,False,False,False,True,False,False,False,False,False,False,False,False,False,False
3,False,True,True,True,True,False,False,False,True,False,False,False,False,False,False,False,False,False,False
4,False,True,True,True,True,False,False,False,True,False,False,False,False,False,False,False,False,False,False


In [3]:
# contar películas por género de la tabla data_clean
movies_genre = pd.read_sql_query(
    """
    SELECT genres, COUNT(*) AS conteo
    FROM data_clean
    GROUP BY genres
    ORDER BY conteo DESC
    """,
    conn,
)
movies_genre.head()

,genres,conteo
0,Comedy,3055
1,Drama,2409
2,Comedy|Romance,2140
3,Action|Adventure|Sci-Fi,1847
4,Crime|Drama,1709


In [45]:
# las 10 películas más vistas por género
movies_genre = pd.read_sql_query(
    """
    SELECT genres, title, COUNT(*) AS conteo
    FROM data_clean
    GROUP BY genres, title
    ORDER BY conteo DESC
    limit 10
    """,
    conn,
)
movies_genre

,genres,title,conteo
0,Comedy|Drama|Romance|War,Forrest Gump (1994),315
1,Crime|Drama,"Shawshank Redemption, The (1994)",302
2,Comedy|Crime|Drama|Thriller,Pulp Fiction (1994),294
3,Action|Sci-Fi|Thriller,"Matrix, The (1999)",267
4,Crime|Horror|Thriller,"Silence of the Lambs, The (1991)",263
5,Action|Adventure|Sci-Fi,Star Wars: Episode IV - A New Hope (1977),241
6,Action|Adventure|Sci-Fi|Thriller,Jurassic Park (1993),229
7,Action|Drama|War,Braveheart (1995),225
8,Action|Sci-Fi,Terminator 2: Judgment Day (1991),217
9,Action|Crime|Drama|Thriller,Fight Club (1999),210


######################################################################
################## 1. sistemas basados en popularidad ###############
#####################################################################

In [69]:
##### recomendaciones basado en popularidad ######
mej_cal=pd.read_sql("""select title, 
            avg(rating) as avg_rat,
            count(*) as view_num
            from data_clean
            where rating<>0
            group by title
            order by avg_rat desc
            limit 10
            """, conn)
mej_cal
# Quitar la columna avg_rat y view_num
mej_cal = mej_cal.drop(['avg_rat', 'view_num'], axis=1)
mej_cal

,title
0,"Shawshank Redemption, The (1994)"
1,Lawrence of Arabia (1962)
2,"Godfather, The (1972)"
3,Dr. Strangelove or: How I Learned to Stop Worr...
4,Patton (1970)
5,Cool Hand Luke (1967)
6,Chinatown (1974)
7,Rear Window (1954)
8,Fight Club (1999)
9,"Princess Bride, The (1987)"


In [71]:
#### las mas vistas con promedio de los que calficaron ###
mas_vis=pd.read_sql("""select title, 
            avg(iif(rating = 0, Null, rating)) as avg_rat,
            count(*) as view_num
            from data_clean
            group by title
            order by view_num desc
            """, conn)

mas_vis.head(15)

# Quitar la columna avg_rat y view_num
mas_vis = mas_vis.drop(['avg_rat', 'view_num'], axis=1)
mas_vis

,title
0,Forrest Gump (1994)
1,"Shawshank Redemption, The (1994)"
2,Pulp Fiction (1994)
3,"Matrix, The (1999)"
4,"Silence of the Lambs, The (1991)"
...,...
856,"Producers, The (1968)"
857,JFK (1991)
858,Bio-Dome (1996)
859,Down Periscope (1996)


In [ ]:
#### las más recomendadas por año de publicacion ###
pd.read_sql("""select year_pub, title, 
            avg(iif(book_rating = 0, Null, book_rating)) as avg_rat,
            count(iif(book_rating = 0, Null, book_rating)) as rat_numb,
            count(*) as read_num
            from full_ratings
            group by  year_pub, book_title
            order by year_pub desc, avg_rat desc limit 20
            """, conn)

,Action,Adventure,Animation,Children,Comedy,Crime,Documentary,Drama,Fantasy,Film-Noir,Horror,IMAX,Musical,Mystery,Romance,Sci-Fi,Thriller,War,Western
0,False,True,True,True,True,False,False,False,True,False,False,False,False,False,False,False,False,False,False
1,False,True,True,True,True,False,False,False,True,False,False,False,False,False,False,False,False,False,False
2,False,True,True,True,True,False,False,False,True,False,False,False,False,False,False,False,False,False,False
3,False,True,True,True,True,False,False,False,True,False,False,False,False,False,False,False,False,False,False
4,False,True,True,True,True,False,False,False,True,False,False,False,False,False,False,False,False,False,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
56214,True,True,True,True,True,False,False,False,False,False,False,False,False,False,False,False,False,False,False
56215,True,True,True,True,True,False,False,False,False,False,False,False,False,False,False,False,False,False,False
56216,True,True,True,True,True,False,False,False,False,False,False,False,False,False,False,False,False,False,False
56217,True,True,True,True,True,False,False,False,False,False,False,False,False,False,False,False,False,False,False


In [41]:
# Agrupar películas por movieId
df_grouped = df.groupby('movieId').agg({'title': 'first', 'genres': 'first', 'rating': 'mean', 'rating_count': 'first'}).reset_index()
df_grouped.tail(20)


,movieId,title,genres,rating,rating_count
841,104841,Gravity (2013),Action|Sci-Fi|IMAX,3.578125,32
842,106782,"Wolf of Wall Street, The (2013)",Comedy|Crime|Drama,3.916667,54
843,108932,The Lego Movie (2014),Action|Adventure|Animation|Children|Comedy|Fan...,3.870968,31
844,109374,"Grand Budapest Hotel, The (2014)",Comedy|Drama,3.784314,51
845,109487,Interstellar (2014),Sci-Fi|IMAX,4.000000,71
846,110102,Captain America: The Winter Soldier (2014),Action|Adventure|Sci-Fi|IMAX,3.741935,31
847,111759,Edge of Tomorrow (2014),Action|Sci-Fi|IMAX,3.977273,44
848,112552,Whiplash (2014),Drama,4.039474,38
849,112556,Gone Girl (2014),Drama|Thriller,3.680556,36
850,112852,Guardians of the Galaxy (2014),Action|Adventure|Sci-Fi,4.050847,59
